# Searching and Downloading Data from the Blue Brain Knowledge Graph using the Knowledge Graph Forge

## Initialize and configure

### Get an authentication token

For now, the [Nexus web application](https://bbp.epfl.ch/nexus/web) can be used to get a token. We are looking for other simpler alternatives.

- Step 1: From the opened web page, click on the login button on the right corner and follow the instructions.

![login-ui](./login-ui.png)

- Step 2: At the end you’ll see a token button on the right corner. Click on it to copy the token.

![login-ui](./copy-token.png)


Once a token is obtained then proceed to paste it below.

In [115]:
import getpass

In [116]:
TOKEN = getpass.getpass()

 ·······································································································································································································································································································································································································································································································································································································································································································································································································································································································································································································································

### Get the knowledge graph schemas

In [78]:
import os 

neuroshapes_dir = "./neuroshapes"
if os.path.exists(neuroshapes_dir):
  !rm -Rf $neuroshapes_dir
! git clone https://github.com/INCF/neuroshapes.git
! cp -R "./neuroshapes/shapes/neurosciencegraph/datashapes/core/dataset" "./neuroshapes/shapes/neurosciencegraph/commons/" 


Cloning into 'neuroshapes'...
remote: Enumerating objects: 127, done.
remote: Counting objects: 100% (127/127), done.
remote: Compressing objects: 100% (84/84), done.
remote: Total 8294 (delta 57), reused 92 (delta 37), pack-reused 8167
Receiving objects: 100% (8294/8294), 7.54 MiB | 374.00 KiB/s, done.
Resolving deltas: 100% (4115/4115), done.


### Configure a client (forge) to access the knowledge graph 

In [118]:
from kgforge.core import KnowledgeGraphForge

In [119]:
# Let target the sscx dissemination project in Nexus
ORG = "public"
PROJECT = "sscx"

In [120]:
config = {
    "Model": {
		"name": "RdfModel",
		"origin": "directory",
		"source": "./neuroshapes/shapes/neurosciencegraph/commons/",
        "context":{
            "iri":"./neuroshapes_context.json"
        }
	},
	"Store": {
        "name": "BlueBrainNexus",
		"endpoint": "https://bbp.epfl.ch/nexus/v1",
        "bucket": f"{ORG}/{PROJECT}",
		"token": TOKEN,
	},
    "Resolvers":{
        "ontology":[{
            "resolver":"OntologyResolver",
            "origin":"store",
            "source":"BlueBrainNexus",
            "targets":[{
                "identifier":"terms",
                "bucket":"neurosciencegraph/datamodels"
            }],
            "result_resource_mapping":"../../../examples/configurations/nexus-resolver/term-to-resource-mapping.hjson"
        }]
    }
    
}


In [121]:
forge = KnowledgeGraphForge(config)

## Search

In [122]:
forge.types()

Managed entity types:
   - Dataset


### For ontologies

#### Set filters

In [181]:
# Supported filters for the time being are:
from kgforge.core.commons.strategies import ResolvingStrategy
text = "somatosensory"
limit=10

In [182]:
# other Search strategy can be ResolvingStrategy.BEST_MATCH, ResolvingStrategy.EXACT_MATCH
brain_region = forge.resolve(text, scope="ontology", target="terms", strategy=ResolvingStrategy.ALL_MATCHES, limit=limit)

In [184]:
forge.as_dataframe(brain_region).head(100)

,id,type,label,isDefinedBy,notation,prefLabel,subClassOf
0,http://api.brain-map.org/api/v2/data/Structure...,Class,"Primary somatosensory area, trunk, layer 1",http://bbp.epfl.ch/neurosciencegraph/ontologie...,SSp-tr1,"Primary somatosensory area, trunk, layer 1",https://neuroshapes.org/BrainRegion
1,http://api.brain-map.org/api/v2/data/Structure...,Class,"Primary somatosensory area, trunk",http://bbp.epfl.ch/neurosciencegraph/ontologie...,SSp-tr,"Primary somatosensory area, trunk",https://neuroshapes.org/BrainRegion
2,http://api.brain-map.org/api/v2/data/Structure...,Class,"Primary somatosensory area, upper limb, layer 6b",http://bbp.epfl.ch/neurosciencegraph/ontologie...,SSp-ul6b,"Primary somatosensory area, upper limb, layer 6b",https://neuroshapes.org/BrainRegion
3,http://api.brain-map.org/api/v2/data/Structure...,Class,"Primary somatosensory area, upper limb",http://bbp.epfl.ch/neurosciencegraph/ontologie...,SSp-ul,"Primary somatosensory area, upper limb",https://neuroshapes.org/BrainRegion
4,http://api.brain-map.org/api/v2/data/Structure...,Class,"Primary somatosensory area, lower limb, layer 1",http://bbp.epfl.ch/neurosciencegraph/ontologie...,SSp-ll1,"Primary somatosensory area, lower limb, layer 1",https://neuroshapes.org/BrainRegion
5,http://api.brain-map.org/api/v2/data/Structure...,Class,"Primary somatosensory area, lower limb",http://bbp.epfl.ch/neurosciencegraph/ontologie...,SSp-ll,"Primary somatosensory area, lower limb",https://neuroshapes.org/BrainRegion
6,http://api.brain-map.org/api/v2/data/Structure...,Class,"Supplemental somatosensory area, layer 4",http://bbp.epfl.ch/neurosciencegraph/ontologie...,SSs4,"Supplemental somatosensory area, layer 4",https://neuroshapes.org/BrainRegion
7,http://api.brain-map.org/api/v2/data/Structure...,Class,Supplemental somatosensory area,http://bbp.epfl.ch/neurosciencegraph/ontologie...,SSs,Supplemental somatosensory area,https://neuroshapes.org/BrainRegion
8,http://api.brain-map.org/api/v2/data/Structure...,Class,"Primary somatosensory area, barrel field, laye...",http://bbp.epfl.ch/neurosciencegraph/ontologie...,SSp-bfd6a,"Primary somatosensory area, barrel field, laye...",https://neuroshapes.org/BrainRegion
9,http://api.brain-map.org/api/v2/data/Structure...,Class,"Primary somatosensory area, barrel field",http://bbp.epfl.ch/neurosciencegraph/ontologie...,SSp-bfd,"Primary somatosensory area, barrel field",https://neuroshapes.org/BrainRegion


### For Morphologies 

#### Set filters

In [108]:
# Supported filters for the time being are:
_type = "ReconstructedCell"
classification_type="nsg:MType"
mType="L4_NBC"
brainRegion = "primary somatosensory cortex"
layer = "layer 4"
encodingFormat="application/swc"
limit=2

#### Run Query

In [109]:
path = forge.paths("Dataset") # to have autocompletion on the properties
data = forge.search(path.type == _type,
                    path.annotation.hasBody.type ==classification_type,
                    path.annotation.hasBody.label ==mType,
                    path.brainLocation.brainRegion.label == brainRegion,
                    path.brainLocation.layer.label == layer,
                    path.distribution.encodingFormat == encodingFormat,
                    limit=limit)

print(str(len(data))+" data of type '"+_type+"' found.")

2 data of type 'ReconstructedCell' found.


#### Display the results

In [110]:
DISPLAY_LIMIT = 10
reshaped_data = forge.reshape(data, keep=["id","name","subject","brainLocation.brainRegion.id","brainLocation.brainRegion.label","brainLocation.layer.id","brainLocation.layer.label", "contribution","brainLocation.layer.id","brainLocation.layer.label","distribution.name","distribution.contentUrl","distribution.encodingFormat"])

forge.as_dataframe(reshaped_data[:DISPLAY_LIMIT])

,id,brainLocation.brainRegion.id,brainLocation.brainRegion.label,brainLocation.layer.id,brainLocation.layer.label,contribution.type,contribution.agent.id,contribution.agent.type,distribution,name,subject.type,subject.species.id,subject.species.label
0,https://bbp.epfl.ch/neurosciencegraph/data/neu...,http://purl.obolibrary.org/obo/UBERON_0008933,primary somatosensory cortex,http://purl.obolibrary.org/obo/UBERON_0005393,layer 4,Contribution,https://www.grid.ac/institutes/grid.5333.6,Agent,[{'contentUrl': 'https://bbp.epfl.ch/neuroscie...,rp110207_L5-2_idB,Subject,http://purl.obolibrary.org/obo/NCBITaxon_10116,Rattus norvegicus
1,https://bbp.epfl.ch/neurosciencegraph/data/neu...,http://purl.obolibrary.org/obo/UBERON_0008933,primary somatosensory cortex,http://purl.obolibrary.org/obo/UBERON_0005393,layer 4,Contribution,https://www.grid.ac/institutes/grid.5333.6,Agent,[{'contentUrl': 'https://bbp.epfl.ch/nexus/v1/...,C200897C-I1,Subject,http://purl.obolibrary.org/obo/NCBITaxon_10116,Rattus norvegicus


In [112]:
print(data[0])

{
    context: https://bbp.neuroshapes.org
    id: https://bbp.epfl.ch/neurosciencegraph/data/neuronmorphologies/09213a3c-5870-4fca-9fbb-7889facbd62a
    type:
    [
        ReconstructedCell
        Dataset
        NeuronMorphology
        Entity
    ]
    annotation:
    {
        type:
        [
            Annotation
            nsg:MTypeAnnotation
        ]
        hasBody:
        {
            id: http://uri.interlex.org/base/ilx_0383215
            type:
            [
                nsg:MType
                Annotation:Body
            ]
            label: L4_NBC
        }
        name: M-type Annotation
    }
    brainLocation:
    {
        type: BrainLocation
        brainRegion:
        {
            id: http://purl.obolibrary.org/obo/UBERON_0008933
            label: primary somatosensory cortex
        }
        layer:
        {
            id: http://purl.obolibrary.org/obo/UBERON_0005393
            label: layer 4
        }
    }
    contribution:
    {
        type: C

#### Dowload

In [97]:
dirpath = "./downloaded/"
forge.download(data, "distribution.contentUrl", dirpath)

### For Trace

#### Set filters

In [98]:
# Supported filters for the time being are:
_type = "Trace"
classification_type="nsg:EType"
eType="cADpyr"
brainRegion = "primary somatosensory cortex"
layer = "layer 5"
encodingFormat="application/nwb"
limit=10

#### Run Query

In [99]:
path = forge.paths("Dataset") # to have autocompletion on the properties
data = forge.search(path.type == _type,
                    path.annotation.hasBody.type ==classification_type,
                    path.annotation.hasBody.label ==eType,
                    path.brainLocation.brainRegion.label == brainRegion,
                    path.brainLocation.layer.label == layer,
                    path.distribution.encodingFormat == encodingFormat,
                    limit=limit)

print(str(len(data))+" data of type '"+_type+"' found.")

10 data of type 'Trace' found.


In [113]:
data[0]

Resource(_last_action=Action(error=None, message=None, operation='search', succeeded=True), _validated=False, _synchronized=True, _store_metadata={'id': 'https://bbp.epfl.ch/neurosciencegraph/data/neuronmorphologies/09213a3c-5870-4fca-9fbb-7889facbd62a', '_self': 'https://bbp.epfl.ch/nexus/v1/resources/public/sscx/datashapes:dataset/neuronmorphologies%2F09213a3c-5870-4fca-9fbb-7889facbd62a', '_constrainedBy': 'nsg:dash/dataset', '_project': 'https://bbp.epfl.ch/nexus/v1/projects/public/sscx', '_rev': 3, '_deprecated': False, '_createdAt': '2020-05-11T08:24:24.233051Z', '_createdBy': 'https://bbp.epfl.ch/nexus/v1/realms/bbp/users/harikris', '_updatedAt': '2020-05-14T08:45:56.716803Z', '_updatedBy': 'https://bbp.epfl.ch/nexus/v1/realms/bbp/users/akkaufma', '_incoming': 'https://bbp.epfl.ch/nexus/v1/resources/public/sscx/datashapes:dataset/neuronmorphologies%2F09213a3c-5870-4fca-9fbb-7889facbd62a/incoming', '_outgoing': 'https://bbp.epfl.ch/nexus/v1/resources/public/sscx/datashapes:datase

#### Display the results

In [100]:
DISPLAY_LIMIT = 10
reshaped_data = forge.reshape(data, keep=["id","name","subject","brainLocation.brainRegion.id","brainLocation.brainRegion.label","brainLocation.layer.id","brainLocation.layer.label", "contribution","brainLocation.layer.id","brainLocation.layer.label","distribution.name","distribution.contentUrl","distribution.encodingFormat"])

forge.as_dataframe(reshaped_data[:DISPLAY_LIMIT])

,id,brainLocation.brainRegion.id,brainLocation.brainRegion.label,brainLocation.layer.id,brainLocation.layer.label,contribution.type,contribution.agent.id,contribution.agent.type,contribution.hadRole.id,contribution.hadRole.label,...,distribution.contentUrl,distribution.encodingFormat,distribution.name,name,subject.type,subject.age.period,subject.age.unitCode,subject.age.value,subject.species.id,subject.species.label
0,https://bbp.epfl.ch/neurosciencegraph/data/882...,http://purl.obolibrary.org/obo/UBERON_0008933,primary somatosensory cortex,http://purl.obolibrary.org/obo/UBERON_0005394,layer 5,Contribution,https://bbp.epfl.ch/neurosciencegraph/data/per...,Agent,http://purl.obolibrary.org/obo/CRO_0000036,data collection role,...,https://bbp.epfl.ch/nexus/v1/files/public/sscx...,application/nwb,C060202A1-SR-C1.nwb,C060202A1-SR-C1,Subject,Post-natal,days,13,http://purl.obolibrary.org/obo/NCBITaxon_10116,Rattus norvegicus
1,https://bbp.epfl.ch/neurosciencegraph/data/42f...,http://purl.obolibrary.org/obo/UBERON_0008933,primary somatosensory cortex,http://purl.obolibrary.org/obo/UBERON_0005394,layer 5,Contribution,https://bbp.epfl.ch/neurosciencegraph/data/per...,Agent,http://purl.obolibrary.org/obo/CRO_0000036,data collection role,...,https://bbp.epfl.ch/nexus/v1/files/public/sscx...,application/nwb,C060110A3-SR-C1.nwb,C060110A3-SR-C1,Subject,Post-natal,days,14,http://purl.obolibrary.org/obo/NCBITaxon_10116,Rattus norvegicus
2,https://bbp.epfl.ch/neurosciencegraph/data/06d...,http://purl.obolibrary.org/obo/UBERON_0008933,primary somatosensory cortex,http://purl.obolibrary.org/obo/UBERON_0005394,layer 5,Contribution,https://bbp.epfl.ch/neurosciencegraph/data/per...,Agent,http://purl.obolibrary.org/obo/CRO_0000036,data collection role,...,https://bbp.epfl.ch/nexus/v1/files/public/sscx...,application/nwb,C060116A6-SR-C1.nwb,C060116A6-SR-C1,Subject,Post-natal,days,14,http://purl.obolibrary.org/obo/NCBITaxon_10116,Rattus norvegicus
3,https://bbp.epfl.ch/neurosciencegraph/data/c57...,http://purl.obolibrary.org/obo/UBERON_0008933,primary somatosensory cortex,http://purl.obolibrary.org/obo/UBERON_0005394,layer 5,Contribution,https://bbp.epfl.ch/neurosciencegraph/data/per...,Agent,http://purl.obolibrary.org/obo/CRO_0000036,data collection role,...,https://bbp.epfl.ch/nexus/v1/files/public/sscx...,application/nwb,C060114A5-SR-C1.nwb,C060114A5-SR-C1,Subject,Post-natal,days,12,http://purl.obolibrary.org/obo/NCBITaxon_10116,Rattus norvegicus
4,https://bbp.epfl.ch/neurosciencegraph/data/a6d...,http://purl.obolibrary.org/obo/UBERON_0008933,primary somatosensory cortex,http://purl.obolibrary.org/obo/UBERON_0005394,layer 5,Contribution,https://bbp.epfl.ch/neurosciencegraph/data/per...,Agent,http://purl.obolibrary.org/obo/CRO_0000036,data collection role,...,https://bbp.epfl.ch/nexus/v1/files/public/sscx...,application/nwb,C060209A3-SR-C1.nwb,C060209A3-SR-C1,Subject,Post-natal,days,13,http://purl.obolibrary.org/obo/NCBITaxon_10116,Rattus norvegicus
5,https://bbp.epfl.ch/neurosciencegraph/data/fab...,http://purl.obolibrary.org/obo/UBERON_0008933,primary somatosensory cortex,http://purl.obolibrary.org/obo/UBERON_0005394,layer 5,Contribution,https://bbp.epfl.ch/neurosciencegraph/data/per...,Agent,http://purl.obolibrary.org/obo/CRO_0000036,data collection role,...,https://bbp.epfl.ch/nexus/v1/files/public/sscx...,application/nwb,C061128B2-SR-C1.nwb,C061128B2-SR-C1,Subject,Post-natal,days,12,http://purl.obolibrary.org/obo/NCBITaxon_10116,Rattus norvegicus
6,https://bbp.epfl.ch/neurosciencegraph/data/59e...,http://purl.obolibrary.org/obo/UBERON_0008933,primary somatosensory cortex,http://purl.obolibrary.org/obo/UBERON_0005394,layer 5,Contribution,https://bbp.epfl.ch/neurosciencegraph/data/per...,Agent,http://purl.obolibrary.org/obo/CRO_0000036,data collection role,...,https://bbp.epfl.ch/nexus/v1/files/public/sscx...,application/nwb,C061128A5-SR-C1.nwb,C061128A5-SR-C1,Subject,Post-natal,days,12,http://purl.obolibrary.org/obo/NCBITaxon_10116,Rattus norvegi

#### Dowload

In [101]:
dirpath = "./downloaded/"
forge.download(data, "distribution.contentUrl", dirpath)

### For LayerThickness 

#### Set filters

In [82]:
# Supported filters for the time being are:
_type = "LayerThickness"
brainRegion = "primary somatosensory cortex"
layer = "layer 2"
encodingFormat="application/xlsx"
limit=10

#### Run query

In [85]:
path = forge.paths("Dataset") # to have autocompletion on the properties
data = forge.search(path.type == _type,
                    #path.brainLocation.layer.label == layer,
                    path.brainLocation.brainRegion.label == brainRegion,
                    path.distribution.encodingFormat == encodingFormat,
                    limit=limit)

print(str(len(data))+" data of type '"+_type+"' found.")

6 data of type 'LayerThickness' found.


#### Display Results

In [86]:
DISPLAY_LIMIT = 10
reshaped_data = forge.reshape(data, keep=["id","name","subject","brainLocation.brainRegion.id","brainLocation.brainRegion.label","brainLocation.layer.id","brainLocation.layer.label", "contribution","brainLocation.layer.id","brainLocation.layer.label","distribution.name","distribution.contentUrl","distribution.encodingFormat"])

forge.as_dataframe(reshaped_data[:DISPLAY_LIMIT])

,id,brainLocation.brainRegion.id,brainLocation.brainRegion.label,brainLocation.layer.id,brainLocation.layer.label,contribution.type,contribution.agent.id,contribution.agent.type,distribution.contentUrl,distribution.encodingFormat,distribution.name,name,subject.type,subject.species.id,subject.species.label
0,https://bbp.epfl.ch/neurosciencegraph/data/thi...,http://purl.obolibrary.org/obo/UBERON_0008933,primary somatosensory cortex,http://purl.obolibrary.org/obo/UBERON_0005390,layer 1,Contribution,https://www.grid.ac/institutes/grid.419043.b,Agent,https://bbp.epfl.ch/neurosciencegraph/data/0e7...,application/xlsx,Cortical layer thickness - layer 1.xlsx,Cortical layer thickness - layer 1,Subject,http://purl.obolibrary.org/obo/NCBITaxon_10116,Rattus norvegicus
1,https://bbp.epfl.ch/neurosciencegraph/data/thi...,http://purl.obolibrary.org/obo/UBERON_0008933,primary somatosensory cortex,http://purl.obolibrary.org/obo/UBERON_0005395,layer 6,Contribution,https://www.grid.ac/institutes/grid.419043.b,Agent,https://bbp.epfl.ch/neurosciencegraph/data/b24...,application/xlsx,Cortical layer thickness - layer 6.xlsx,Cortical layer thickness - layer 6,Subject,http://purl.obolibrary.org/obo/NCBITaxon_10116,Rattus norvegicus
2,https://bbp.epfl.ch/neurosciencegraph/data/thi...,http://purl.obolibrary.org/obo/UBERON_0008933,primary somatosensory cortex,http://purl.obolibrary.org/obo/UBERON_0005393,layer 4,Contribution,https://www.grid.ac/institutes/grid.419043.b,Agent,https://bbp.epfl.ch/neurosciencegraph/data/ec9...,application/xlsx,Cortical layer thickness - layer 4.xlsx,Cortical layer thickness - layer 4,Subject,http://purl.obolibrary.org/obo/NCBITaxon_10116,Rattus norvegicus
3,https://bbp.epfl.ch/neurosciencegraph/data/thi...,http://purl.obolibrary.org/obo/UBERON_0008933,primary somatosensory cortex,http://purl.obolibrary.org/obo/UBERON_0005394,layer 5,Contribution,https://www.grid.ac/institutes/grid.419043.b,Agent,https://bbp.epfl.ch/neurosciencegraph/data/565...,application/xlsx,Cortical layer thickness - layer 5.xlsx,Cortical layer thickness - layer 5,Subject,http://purl.obolibrary.org/obo/NCBITaxon_10116,Rattus norvegicus
4,https://bbp.epfl.ch/neurosciencegraph/data/thi...,http://purl.obolibrary.org/obo/UBERON_0008933,primary somatosensory cortex,http://purl.obolibrary.org/obo/UBERON_0005391,layer 2,Contribution,https://www.grid.ac/institutes/grid.419043.b,Agent,https://bbp.epfl.ch/neurosciencegraph/data/f73...,application/xlsx,Cortical layer thickness - layer 2.xlsx,Cortical layer thickness - layer 2,Subject,http://purl.obolibrary.org/obo/NCBITaxon_10116,Rattus norvegicus
5,https://bbp.epfl.ch/neurosciencegraph/data/thi...,http://purl.obolibrary.org/obo/UBERON_0008933,primary somatosensory cortex,http://purl.obolibrary.org/obo/UBERON_0005392,layer 3,Contribution,https://www.grid.ac/institutes/grid.419043.b,Agent,https://bbp.epfl.ch/neurosciencegraph/data/315...,application/xlsx,Cortical layer thickness - layer 3.xlsx,Cortical layer thickness - layer 3,Subject,http://purl.obolibrary.org/obo/NCBITaxon_10116,Rattus norvegicus


#### Dowload

In [88]:
dirpath = "./downloaded/"
forge.download(data, "distribution.contentUrl", dirpath)

### For Neuron Density 

#### Set filters

In [102]:
# Supported filters for the time being are:
_type = "NeuronDensity"
brainRegion = "primary somatosensory cortex"
layer = "layer 2"
encodingFormat="application/xlsx"
limit=10

#### Run query

In [104]:
path = forge.paths("Dataset") # to have autocompletion on the properties
data = forge.search(path.type == _type,
                    #path.brainLocation.layer.label == layer,
                    path.brainLocation.brainRegion.label == brainRegion,
                    path.distribution.encodingFormat == encodingFormat,
                    limit=limit)

print(str(len(data))+" data of type '"+_type+"' found.")

<action> _wrap_create_connection
<error> ClientConnectorError: Cannot connect to host bbp.epfl.ch:443 ssl:default [Connect call failed ('192.33.211.24', 443)]



TypeError: object of type 'NoneType' has no len()

#### Display Results

In [92]:
DISPLAY_LIMIT = 10
reshaped_data = forge.reshape(data, keep=["id","name","subject","brainLocation.brainRegion.id","brainLocation.brainRegion.label","brainLocation.layer.id","brainLocation.layer.label", "contribution","brainLocation.layer.id","brainLocation.layer.label","distribution.name","distribution.contentUrl","distribution.encodingFormat"])

forge.as_dataframe(reshaped_data[:DISPLAY_LIMIT])

,id,brainLocation.brainRegion.id,brainLocation.brainRegion.label,brainLocation.layer.id,brainLocation.layer.label,contribution.type,contribution.agent.id,contribution.agent.type,distribution.contentUrl,distribution.encodingFormat,distribution.name,name,subject.type,subject.species.id,subject.species.label
0,https://bbp.epfl.ch/neurosciencegraph/data/den...,http://purl.obolibrary.org/obo/UBERON_0008933,primary somatosensory cortex,http://purl.obolibrary.org/obo/UBERON_0005393,layer 4,Contribution,https://www.grid.ac/institutes/grid.419043.b,Agent,https://bbp.epfl.ch/neurosciencegraph/data/fa7...,application/xlsx,Cortical neuron density - layer 4.xlsx,Cortical neuron density - layer 4,Subject,http://purl.obolibrary.org/obo/NCBITaxon_10116,Rattus norvegicus
1,https://bbp.epfl.ch/neurosciencegraph/data/den...,http://purl.obolibrary.org/obo/UBERON_0008933,primary somatosensory cortex,http://purl.obolibrary.org/obo/UBERON_0005394,layer 5,Contribution,https://www.grid.ac/institutes/grid.419043.b,Agent,https://bbp.epfl.ch/neurosciencegraph/data/2dd...,application/xlsx,Cortical neuron density - layer 5.xlsx,Cortical neuron density - layer 5,Subject,http://purl.obolibrary.org/obo/NCBITaxon_10116,Rattus norvegicus
2,https://bbp.epfl.ch/neurosciencegraph/data/den...,http://purl.obolibrary.org/obo/UBERON_0008933,primary somatosensory cortex,http://purl.obolibrary.org/obo/UBERON_0005391,layer 2,Contribution,https://www.grid.ac/institutes/grid.419043.b,Agent,https://bbp.epfl.ch/neurosciencegraph/data/ecc...,application/xlsx,Cortical neuron density - layer 2.xlsx,Cortical neuron density - layer 2,Subject,http://purl.obolibrary.org/obo/NCBITaxon_10116,Rattus norvegicus
3,https://bbp.epfl.ch/neurosciencegraph/data/den...,http://purl.obolibrary.org/obo/UBERON_0008933,primary somatosensory cortex,http://purl.obolibrary.org/obo/UBERON_0005392,layer 3,Contribution,https://www.grid.ac/institutes/grid.419043.b,Agent,https://bbp.epfl.ch/neurosciencegraph/data/42e...,application/xlsx,Cortical neuron density - layer 3.xlsx,Cortical neuron density - layer 3,Subject,http://purl.obolibrary.org/obo/NCBITaxon_10116,Rattus norvegicus
4,https://bbp.epfl.ch/neurosciencegraph/data/den...,http://purl.obolibrary.org/obo/UBERON_0008933,primary somatosensory cortex,http://purl.obolibrary.org/obo/UBERON_0005390,layer 1,Contribution,https://www.grid.ac/institutes/grid.419043.b,Agent,https://bbp.epfl.ch/neurosciencegraph/data/e60...,application/xlsx,Cortical neuron density - layer 1.xlsx,Cortical neuron density - layer 1,Subject,http://purl.obolibrary.org/obo/NCBITaxon_10116,Rattus norvegicus
5,https://bbp.epfl.ch/neurosciencegraph/data/den...,http://purl.obolibrary.org/obo/UBERON_0008933,primary somatosensory cortex,http://purl.obolibrary.org/obo/UBERON_0005395,layer 6,Contribution,https://www.grid.ac/institutes/grid.419043.b,Agent,https://bbp.epfl.ch/neurosciencegraph/data/bc3...,application/xlsx,Cortical neuron density - layer 6.xlsx,Cortical neuron density - layer 6,Subject,http://purl.obolibrary.org/obo/NCBITaxon_10116,Rattus norvegicus


#### Dowload

In [94]:
dirpath = "./downloaded/"
forge.download(data, "distribution.contentUrl", dirpath)

<action> send
<error> ConnectionError: HTTPSConnectionPool(host='bbp.epfl.ch', port=443): Max retries exceeded with url: /nexus/v1/files/public/sscx/ecca604e-759a-46f1-a61b-9992a7576e00 (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x120590110>: Failed to establish a new connection: [Errno 60] Operation timed out'))



### For Atlas Release

In [14]:
# Let target the bbp/atlas project in Nexus

forge_atlas = KnowledgeGraphForge(config, bucket="bbp/atlas")

Atlas related types:
                AtlasRelease
                CellPositions
                BrainParcellationDataLayer
                CellDensityDataLayer
                GeneExpressionVolumetricDataLayer
                GliaCellDensity
                NISSLImageDataLayer

#### Set filters

In [13]:
# Supported filters for the time being are:
_type = "CellPositions"
limit=10

#### Run query

In [ ]:
path = forge_atlas.paths("Dataset") # to have autocompletion on the properties
data = forge_atlas.search(path.type == _type,
                    limit=limit)

print(str(len(data))+" data of type '"+_type+"' found.")

#### Display Results

In [ ]:
DISPLAY_LIMIT = 10
reshaped_data = forge_atlas.reshape(data, keep=["id","name","subject","brainLocation.brainRegion.id","brainLocation.brainRegion.label","brainLocation.layer.id","brainLocation.layer.label", "contribution","brainLocation.layer.id","brainLocation.layer.label","distribution.name","distribution.contentUrl","distribution.encodingFormat"])

forge.as_dataframe(reshaped_data[:DISPLAY_LIMIT])

In [ ]:
dirpath = "./downloaded/"
forge.download(data, "distribution.contentUrl", dirpath)